<a href="https://colab.research.google.com/github/samtam0714/Chicago-Food-Inspection-Analytics/blob/main/food_inspection_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas

     |████████████████████████████████| 972kB 2.9MB/s 
     |████████████████████████████████| 10.9MB 10.0MB/s 
     |████████████████████████████████| 14.7MB 303kB/s 


In [ ]:
# importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os 
import seaborn as sns
import geopandas as gpd
import folium
from folium import plugins
import datetime
import math

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#load dataset
dataset = pd.read_csv('/content/drive/My Drive/Food_Inspections.csv')
dataset.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2385628,ALL IN 1 DISCOUNT INC,ALL IN 1 DISCOUNT INC,2744661.0,Grocery Store,Risk 3 (Low),5719-5725 W DIVERSEY AVE,CHICAGO,IL,60639.0,08/03/2020,License,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.931121,-87.769554,"(-87.76955418373417, 41.9311212237607)"
1,2385591,HELEN'S GROCERY & LIQ,HELEN'S GROCERY & LIQ,8017.0,Grocery Store,Risk 3 (Low),2300 W 21ST ST,CHICAGO,IL,60608.0,07/31/2020,Canvass,Out of Business,NaN,41.854005,-87.683363,"(-87.6833629225202, 41.8540047775837)"
2,2385590,PETE'S FRESH SUPERMARKET,PETE'S FRESH SUPERMARKET,1771242.0,Grocery Store,Risk 1 (High),2526 W CERMAK RD,CHICAGO,IL,60608.0,07/31/2020,Complaint Re-Inspection,Pass,NaN,41.852101,-87.689051,"(-87.68905111060724, 41.85210065324028)"
3,2385587,QUEST FOOD MANAGEMENT SERVICES INC,QUEST FOOD MANAGEMENT SERVICES INC,2744583.0,NaN,Risk 1 (High),3300 N CAMPBELL AVE,CHICAGO,IL,60618.0,07/31/2020,License,Not Ready,NaN,41.941426,-87.690799,"(-87.69079930080058, 41.94142629278533)"
4,2385543,OUTLET KINGDOM,NaN,2054475.0,GENERAL STORE,Risk 3 (Low),3449-3451 S HALSTED ST,CHICAGO,IL,60608.0,07/30/2020,Canvass,Out of Business,NaN,41.831075,-87.645971,"(-87.6459711121892, 41.831074943653626)"


In [ ]:
dataset = dataset.applymap(lambda s:s.lower() if type(s) == str else s)
dataset.head(2)

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2385628,all in 1 discount inc,all in 1 discount inc,2744661.0,grocery store,risk 3 (low),5719-5725 w diversey ave,chicago,il,60639.0,08/03/2020,license,pass w/ conditions,5. procedures for responding to vomiting and d...,41.931121,-87.769554,"(-87.76955418373417, 41.9311212237607)"
1,2385591,helen's grocery & liq,helen's grocery & liq,8017.0,grocery store,risk 3 (low),2300 w 21st st,chicago,il,60608.0,07/31/2020,canvass,out of business,NaN,41.854005,-87.683363,"(-87.6833629225202, 41.8540047775837)"


In [ ]:
dataset.shape

(208925, 17)

In [ ]:
list(dataset.columns) 

['Inspection ID',
 'DBA Name',
 'AKA Name',
 'License #',
 'Facility Type',
 'Risk',
 'Address',
 'City',
 'State',
 'Zip',
 'Inspection Date',
 'Inspection Type',
 'Results',
 'Violations',
 'Latitude',
 'Longitude',
 'Location']

In [ ]:
duplicaterow = dataset.duplicated().any() # False
duplicaterow = dataset.duplicated(subset=[
 'DBA Name',
 'AKA Name',
 'Facility Type',
 'Risk',
 'Address',
 'City',
 'State',
 'Zip',
 'Inspection Date',
 'Inspection Type',
 'Results',
 'Violations',
 'Latitude',
 'Longitude']).any()

In [ ]:
duplicaterow

True

In [ ]:
df = dataset[dataset.duplicated([
 'DBA Name',
 'AKA Name',
 'Facility Type',
 'Risk',
 'Address',
 'City',
 'State',
 'Zip',
 'Inspection Date',
 'Inspection Type',
 'Results',
 'Violations',
 'Latitude',
 'Longitude',
 ])]

In [ ]:
df=dataset.drop_duplicates(subset=[
 'DBA Name',
 'AKA Name',
 'Facility Type',
 'Risk',
 'Address',
 'City',
 'State',
 'Zip',
 'Inspection Date',
 'Inspection Type',
 'Results',
 'Violations',
 'Latitude',
 'Longitude'])
df.head()

,Inspection ID,DBA Name,AKA Name,License #,Facility Type,Risk,Address,City,State,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,2385628,all in 1 discount inc,all in 1 discount inc,2744661.0,grocery store,risk 3 (low),5719-5725 w diversey ave,chicago,il,60639.0,08/03/2020,license,pass w/ conditions,5. procedures for responding to vomiting and d...,41.931121,-87.769554,"(-87.76955418373417, 41.9311212237607)"
1,2385591,helen's grocery & liq,helen's grocery & liq,8017.0,grocery store,risk 3 (low),2300 w 21st st,chicago,il,60608.0,07/31/2020,canvass,out of business,NaN,41.854005,-87.683363,"(-87.6833629225202, 41.8540047775837)"
2,2385590,pete's fresh supermarket,pete's fresh supermarket,1771242.0,grocery store,risk 1 (high),2526 w cermak rd,chicago,il,60608.0,07/31/2020,complaint re-inspection,pass,NaN,41.852101,-87.689051,"(-87.68905111060724, 41.85210065324028)"
3,2385587,quest food management services inc,quest food management services inc,2744583.0,NaN,risk 1 (high),3300 n campbell ave,chicago,il,60618.0,07/31/2020,license,not ready,NaN,41.941426,-87.690799,"(-87.69079930080058, 41.94142629278533)"
4,2385543,outlet kingdom,NaN,2054475.0,general store,risk 3 (low),3449-3451 s halsted st,chicago,il,60608.0,07/30/2020,canvass,out of business,NaN,41.831075,-87.645971,"(-87.6459711121892, 41.831074943653626)"


In [ ]:
df.shape

(206681, 17)

In [ ]:
df = df.drop('Inspection ID',axis=1)
df = df.drop('AKA Name',axis=1)
df = df.drop('City',axis=1)
df = df.drop('State',axis=1)

In [ ]:
df.dtypes

DBA Name            object
License #          float64
Facility Type       object
Risk                object
Address             object
Zip                float64
Inspection Date     object
Inspection Type     object
Results             object
Violations          object
Latitude           float64
Longitude          float64
Location            object
dtype: object

In [ ]:
df.isnull().sum()

DBA Name               0
License #             17
Facility Type       4656
Risk                  72
Address                0
Zip                   51
Inspection Date        0
Inspection Type        1
Results                0
Violations         53423
Latitude             675
Longitude            675
Location             675
dtype: int64

In [ ]:
df = df.dropna(how='any', subset=['License #','Zip','Risk','Inspection Type','Latitude','Longitude','Location'])
df.isnull().sum()

DBA Name               0
License #              0
Facility Type       4583
Risk                   0
Address                0
Zip                    0
Inspection Date        0
Inspection Type        0
Results                0
Violations         53141
Latitude               0
Longitude              0
Location               0
dtype: int64

In [ ]:
df['Facility Type'] = df['Facility Type'].fillna('None')
df['Violations'] = df['Violations'].fillna('None')

In [ ]:
data = df[(df.Results != 'business not located') & (df.Results != 'no entry') & (df.Results != 'not ready') & (df.Results != 'out of business')]

In [ ]:
data

,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,all in 1 discount inc,2744661.0,grocery store,risk 3 (low),5719-5725 w diversey ave,60639.0,08/03/2020,license,pass w/ conditions,5. procedures for responding to vomiting and d...,41.931121,-87.769554,"(-87.76955418373417, 41.9311212237607)"
2,pete's fresh supermarket,1771242.0,grocery store,risk 1 (high),2526 w cermak rd,60608.0,07/31/2020,complaint re-inspection,pass,None,41.852101,-87.689051,"(-87.68905111060724, 41.85210065324028)"
5,chop suey huang,2463091.0,restaurant,risk 1 (high),5202 n lincoln ave,60625.0,07/30/2020,canvass re-inspection,pass w/ conditions,5. procedures for responding to vomiting and d...,41.975976,-87.692494,"(-87.6924942225335, 41.97597608993619)"
6,odyssey chicago river,2713082.0,restaurant,risk 1 (high),455 n cityfront plaza dr,60611.0,07/27/2020,license,pass,None,41.890001,-87.621439,"(-87.62143868830125, 41.890001335214684)"
7,brando's,2738736.0,restaurant,risk 2 (medium),343 s dearborn st,60604.0,07/28/2020,license,pass,None,41.877147,-87.629121,"(-87.62912123316899, 41.877146786216336)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
208920,t tasty house,2017359.0,restaurant,risk 1 (high),2002 s wentworth ave,60616.0,02/10/2010,license re-inspection,pass,None,41.855282,-87.631993,"(-87.63199264229414, 41.85528170377401)"
208921,wrigleyville dog,25051.0,restaurant,risk 1 (high),3735 n clark st,60613.0,02/02/2010,canvass,fail,3. potentially hazardous food meets temperatur...,41.949895,-87.658511,"(-87.65851083690285, 41.94989452098964)"
208922,adler school inc,6765.0,daycare (2 - 6 years),risk 1 (high),2237-2239 w lawrence ave,60625.0,01/22/2010,license re-inspection,pass,None,41.968554,-87.685483,"(-87.68548253142126, 41.968553831007675)"
208923,best sub no2,1647305.0,restaurant,risk 2 (medium),3426 w diversey ave,60647.0,02/10/2010,canvass,pass,33. food and non-food contact equipment utensi...,41.932069,-87.713294,"(-87.71329441237397, 41.932068626464286)"


In [ ]:
cleanup_nums = {"Risk":     {"risk 1 (high)": 2, "risk 2 (medium)": 1, "risk 3 (low)" : 0},
                "Results": {"pass": 0, "pass w/ conditions": 1, "fail": 2 }}

In [ ]:
data.replace(cleanup_nums, inplace=True)
data.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6666: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regex=regex,


,DBA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Results,Violations,Latitude,Longitude,Location
0,all in 1 discount inc,2744661.0,grocery store,0,5719-5725 w diversey ave,60639.0,08/03/2020,license,1,5. procedures for responding to vomiting and d...,41.931121,-87.769554,"(-87.76955418373417, 41.9311212237607)"
2,pete's fresh supermarket,1771242.0,grocery store,2,2526 w cermak rd,60608.0,07/31/2020,complaint re-inspection,0,None,41.852101,-87.689051,"(-87.68905111060724, 41.85210065324028)"
5,chop suey huang,2463091.0,restaurant,2,5202 n lincoln ave,60625.0,07/30/2020,canvass re-inspection,1,5. procedures for responding to vomiting and d...,41.975976,-87.692494,"(-87.6924942225335, 41.97597608993619)"
6,odyssey chicago river,2713082.0,restaurant,2,455 n cityfront plaza dr,60611.0,07/27/2020,license,0,None,41.890001,-87.621439,"(-87.62143868830125, 41.890001335214684)"
7,brando's,2738736.0,restaurant,1,343 s dearborn st,60604.0,07/28/2020,license,0,None,41.877147,-87.629121,"(-87.62912123316899, 41.877146786216336)"


In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
data.to_csv('data.csv')
!cp data.csv "drive/My Drive/"